<a href="https://colab.research.google.com/github/readall/indiatranslator/blob/main/indicTranslate_Transliter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
rm -rf sample_data
rm -rf fairseq
rm -rf indicTrans
rm indic-en.zip

In [2]:
%%bash
rm -rf sample_data
pip install tqdm boto3 requests regex sentencepiece sacremoses transformers easyocr
pip install mosestokenizer indic-nlp-library mock sacrebleu tensorboardX pyarrow subword-nmt
git clone --depth=1 https://github.com/pytorch/fairseq.git
cd fairseq
pip install --editable ./
%cd ..
# pip install fairseq


Obtaining file:///content/fairseq
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Running setup.py develop for fairseq


Cloning into 'fairseq'...
bash: line 7: fg: no job control


In [3]:
# clone the repo for running evaluation
!git clone --depth=1 https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone --depth=1 https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone --depth=1 https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone --depth=1 https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 45 (delta 9), reused 14 (delta 1), pack-reused 0
Unpacking objects: 100% (45/45), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 77 (delta 5), reused 46 (delta 1), pack-reused 0
Unpacking objects: 100% (77/77), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 33 (delta 3), reused 18 (delta 1), pack-reused 0
Unpacking objects: 100% (33/33), done.
Checking out files: 100% (28/28), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compre

In [4]:
%%bash
# downloading the indic-indic model
wget https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
unzip indic-en.zip
rm indic-en.zip
wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
unzip m2m.zip
rm m2m.zip

# %cd indicTrans

Archive:  indic-en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/preprocess.log  
  inflating: indic-en/final_bin/dict.TGT.txt  
  inflating: indic-en/final_bin/test.SRC-TGT.SRC.idx  
  inflating: indic-en/final_bin/test.SRC-TGT.TGT.idx  
  inflating: indic-en/final_bin/train.SRC-TGT.SRC.idx  
  inflating: indic-en/final_bin/dict.SRC.txt  
  inflating: indic-en/final_bin/valid.SRC-TGT.TGT.idx  
  inflating: indic-en/final_bin/test.SRC-TGT.TGT.bin  
  inflating: indic-en/final_bin/valid.SRC-TGT.TGT.bin  
  inflating: indic-en/final_bin/train.SRC-TGT.TGT.idx  
  inflating: indic-en/final_bin/train.SRC-TGT.TGT.bin  
  inflating: indic-en/final_bin/valid.SRC-TGT.SRC.idx  
  inflating: indic-en/final_bin/train.SRC-TGT.SRC.bin  
  inflating: ind

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
!pwd
!cp /content/m2m/vocab/bpe_codes.32k.SRC_TGT /content/m2m/vocab/bpe_codes.32k.SRC


/content/indicTrans


In [7]:
%%bash
apt-get install python-numpy libicu-dev
pip install pyicu pycld2 morfessor 
pip install polyglot


Reading package lists...
Building dependency tree...
Reading state information...
python-numpy is already the newest version (1:1.13.3-2ubuntu1).
libicu-dev is already the newest version (60.2-3ubuntu3.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [1]:
!polyglot download embeddings2.en transliteration2.hi
!polyglot download transliteration2.kn

[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data]   Package embeddings2.en is already up-to-date!
[polyglot_data] Downloading package transliteration2.hi to
[polyglot_data]     /root/polyglot_data...
[polyglot_data] Downloading package transliteration2.kn to
[polyglot_data]     /root/polyglot_data...
[polyglot_data]   Package transliteration2.kn is already up-to-date!


In [3]:
# we need to be in indicTrans directory for translation model to load
%pwd
%cd
%cd /content/indicTrans/
%pwd

/root
/content/indicTrans


'/content/indicTrans'

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt  
import easyocr
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from indicTrans.inference.engine import Model
from polyglot.text import Text

class KannadaToHindi :
  def __init__(self):
    self.reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    self.indicm2m_model = Model(expdir='/content/m2m')


  def getTextFromImage(self, image_file):
    reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory
    result = reader.readtext(image_file, detail = 0)
    return result

  def cleanTransliterText(self, transliter_text):
    blank = ['',"", " ", ' ', '\t']
    s_list = []
    for x in transliter_text:
      if x not in blank:
        s_list.append(x)
    return s_list

  def getTransliterredText(self,text_from_image):
    image_text = str(text_from_image)
    text = Text(image_text)
    transliter_text = text.transliterate("hi")
    trans_str = " ".join(self.cleanTransliterText(transliter_text))
    return trans_str

  def TranslateKannadaImageToHindi(self, kannada_img_file):
    text_from_image = self.reader.readtext(kannada_img_file, detail = 0)
    transliterred_result = self.getTransliterredText(text_from_image)
    translated_result = self.indicm2m_model.batch_translate(text_from_image, 'kn', 'hi')
    return translated_result, transliterred_result


In [5]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_01.jpg'
transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

Initializing vocab and bpe
Initializing model for translation


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 21/21 [00:00<00:00, 19744.54it/s]
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


होनदीसि बरेािरि कापादु हनि बुददीवंट सनरकसहिसु मले ासन बेट रामकृष्ण गुदद कोटिरुव सरियाद ारीसि बिट सथल्डलली बरेािरि कुवेनपु ापपुगे पंडितरु त मले कृष्णदेवराान ासथॉहदलली िददरु निरिन मुल त
['कृपया इस प्रकार लिखेंः', 'सुरक्षित रखें।', 'बूंद-बूंद', '2 में से', 'बुद्धिमान व्यक्ति', 'इसे सहेजें', 'बारिश', 'सीटों की संख्या', '4 नंबर', 'पर्वत', 'राम कृष्ण', 'पीठ', 'पर्वतमाला', 'सही जवाब चुनने के लिए नीचे दिए गए पते पर लिखेंः', '(शानदार, अप्पू, पंडित,', 'बारिश होती है।', '१. संपर्क', 'श्री कृष्णदेवराय के दरबार में', 'वे मौजूद थे।', 'जल स्रोत', 'संस्कृत']


In [6]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_02.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

100%|██████████| 9/9 [00:00<00:00, 8149.55it/s]


केुतिरुव सरियाद ारीसि खअली बिट सथल्डलली बरेािरि कुवेनपु ापपुगे पंडितरु मले क ृष्णदेवराान ासथॉहदलली पंडितरु िददरु निरिन मुल मले कनंडामम न हरके पडीावननु बरेदवरु कुवेनपु ालिंगन पदद ार्थ ापपुगे
['परिसर में दी गई सही जानकारी', 'जवाब चुनें और खाली स्थान पर लिखेंः', '(शानदार, अप्पू, पंडित, संस्कृत, वर्षा)', '1. क.', 'विद्वान भगवान वेंकटेश्वर के दरबार में मौजूद थे।', 'जल स्रोत वर्षा', '3 नहीं', 'उन्होंने कहा कि कन्नड़ कविता की रचना बेहतरीन है।', 'आलिंगन शब्द का अर्थ है कर्ज लेना।']


In [7]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_03.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

100%|██████████| 16/16 [00:00<00:00, 16644.06it/s]


वाकयादलली बिसिल निरु निरु ावियााितु मोद ावाग भारवााितु ावि तनिदु भारवााितु विदासागरनोंडिगे वाद मदलु ारु विदासागरनोंडिगे वाद मदलु टेनाली रामकृष्ण टेनाली रामकृष्ण वादकके ाव गरनथवननु ारीसिकोंडनु रामकृष्ण वादकके महिशबंधन ेनब गरनथवननु ारीसिकोंडनु कनंडद कंड ावउदननु मरेाबारदु कंड कनंडवननु मरेाबारदु
['एक वाक्य में', 'गर्म पानी का क्या हुआ?', 'गर्मी का पानी भाप बन गया।', '2 में से', 'बादल कब भारी हुआ?', 'उष्मायन वातावरण ठंडा हो गया।', 'कौन विद्यासागर से बहस करने के लिए सहमत हुआ?', 'उत्तरः -', 'तेनाली रामकृष्ण विद्यासागर के साथ बहस करने के लिए सहमत हुए', '4 टेनाली', 'रामकृष्ण ने किस ग्रंथ का चयन किया?', 'उन्होंने उतेनाली रामकृष्ण वाद के लिए तिलकस्थ महिषाबंधन नामक पुस्तक का चयन किया।', 'यह संख्या 5 है', 'हिंदी के प्रति हमें क्या याद रखना चाहिए?', 'कन्नड़ भाषा को न भूलेंः', 'जवाब दीजिएः']


In [8]:
kannada_img_file = '/content/drive/MyDrive/Colab Notebooks/kannada_04.jpg'
# transcode = KannadaToHindi()
translated, translittered = transcode.TranslateKannadaImageToHindi(kannada_img_file)
print(translittered)
print(translated)

100%|██████████| 28/28 [00:00<00:00, 10266.68it/s]


प्रशनेगलिगे ेरदु मुरु वाकयादलली विदासागरन मुख ारलितु रामकृष्ण विदासागरनोंडिगे वाद मदलु करना विदासागरन मुख ारलितु महिशबंडन ेंडरेनु ेंडरे ेलेलिन कददीगलु महिशबंधन ेंडरे ेाममे कटुव हगग िदे महिशबंधंड ार्थ कोटिरुव मैननु ारु ारिगे हेलिदरु विशा मोदले तिलिदिददरे नानु िलिगे बरुटले मैननु विदासागर कृष्णदेवराानिगे हेलिदनु िदु नाममुरिन दंकाौव हुदुगरिगु गोटु मैननु टेनाली रामकृष्ण विदासागरनिगे हेलिदनु केलगिन कविगल परिकहा मादी कुवेनपु पुरना हेसरु कुपपलेली वेंकतपप पुटपप जनन सथल सहिवमोगग जीललेा कुपपलेली िरलिल
['नीचे दिए गए प्रश्नों के लिए दो/तीन पंक्ति में', 'विद्यासागर का चेहरा क्यों फूला?', 'उथेनाली रामकृष्ण विद्यासागर से बहस करने के लिए सहमत हुई', 'विद्यासागर का चेहरा फूट पड़ा', 'तिलकस्थ महिषाबंदन क्या है?', '0:', 'तिलकाष्ठा', '"" "" "" "बैल की छड़ियां" "" "और" "" "महिषबंधन" "" "यानि बैल की रस्सी" ""', 'तिलकत्था महिषाबंधन का अर्थ', 'ग्यारहवीं का वादा किसने किया?', 'यह विषय पहले से है', 'अगर मैं जानता हूं तो यहां आऊंगा।', 'जवाबः इस', 'विद्यासागर ने कृष्णदेवराय से कहा था -', 'हमारे पशुपालक भी यह जानते ह

In [ ]:
# from polyglot.downloader import downloader
# print(downloader.supported_languages_table("transliteration2"))